In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-aiml-project2")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: nahmed30-azureml-workspace
Azure region: centralus
Subscription id: 16bc73b5-82be-47f2-b5ab-f2373344794c
Resource group: epe-poc-nazeer


In [2]:

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cpu-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Verify that cluster does not exist already
try:
       cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
       print('Found existing cluster, use it.')
except ComputeTargetException:
       compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
       cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.core import Environment

# sklearn_env = Environment.from_conda_specification(name="MyAzureML-Training", file_path = "conda_dependencies.yml")

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.randomparametersampling?view=azure-ml-py

# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/scikit-learn/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.ipynb

ps = RandomParameterSampling( {
    "--C": uniform(0.1, 0.4),
    "--max_iter": choice(50, 100, 200, 250) 
    }
)

print(ps.__str__)

<method-wrapper '__str__' of RandomParameterSampling object at 0x7f2d80193490>


In [5]:
# Specify a Policy
# https://azure.github.io/azureml-sdk-for-r/reference/bandit_policy.html
# In this example, the early termination policy is applied at every interval
# when metrics are reported, starting at evaluation interval 5. Any run whose
# best metric is less than (1 / (1 + 0.1)) or 91\% of the best performing run will
# be terminated

policy = BanditPolicy(slack_factor = 0.2,
                    evaluation_interval = 1,
                    delay_evaluation = 5)


if "training" not in os.listdir():
    os.mkdir("./training")

In [6]:
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path = 'conda_dependencies.yml')

In [7]:
from azureml.core import ScriptRunConfig

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='trainudacityazureml.py',
                      compute_target='cpu-cluster',
                      environment=sklearn_env)


In [8]:
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# https://azure.github.io/azureml-sdk-for-r/reference/hyperdrive_config.html
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py
hyperdrive_config =  HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, # Define the primary metric goal 
                                max_total_runs=10,
                                max_concurrent_runs=4)

In [9]:
run = exp.submit(config=hyperdrive_config)

In [10]:
run.wait_for_completion(show_output=True)

RunId: HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6
Web View: https://ml.azure.com/runs/HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6?wsid=/subscriptions/16bc73b5-82be-47f2-b5ab-f2373344794c/resourcegroups/epe-poc-nazeer/workspaces/nahmed30-azureml-workspace&tid=db05faca-c82a-4b9d-b9c5-0f64b6755421

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-06-08T17:47:38.764868][API][INFO]Experiment created<END>\n""<START>[2022-06-08T17:47:39.650475][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2022-06-08T17:47:40.2667561Z][SCHEDULER][INFO]Scheduling job, id='HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6_0'<END><START>[2022-06-08T17:47:40.4365024Z][SCHEDULER][INFO]Scheduling job, id='HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6_1'<END><START>[2022-06-08T17:47:40.5198647Z][SCHEDULER][INFO]Scheduling job, id='HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6_2'<END>"<START>[2022-06-08T17:47:40.499698][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the 

{'runId': 'HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-06-08T17:47:38.519419Z',
 'endTimeUtc': '2022-06-08T17:55:11.824688Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '60560ca2-ae8a-4745-b893-ce980d593ca7',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1077-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.41.0',
  'space_size': 'infinite_space_size',
  'score': '0.9153920854576354',
  'best_child_run_id': 'HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://nahmed30storage.blob.cor

In [11]:
# use get_status() to get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 2, 'idleNodeCount': 2, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-06-08T17:49:39.431000+00:00', 'errors': None, 'creationTime': '2022-05-25T03:10:05.547230+00:00', 'modifiedTime': '2022-05-25T03:10:09.035389+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS12_V2'}


In [12]:
# Get your best run and save the model from that run.

In [13]:
best_run = run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-aiml-project2,HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(parameter_values )

['--C', '0.2744966601103044', '--max_iter', '200']


In [15]:
# model_name = "FAQ_ranker"
# model = Model(ws, name=model_name)
print(best_run.get_details()['runDefinition']['arguments'])
# print(best_run.get_details())
# print(best_run.get_details()['runId'])


print("*******************************************************************************")
print(best_run.log_list)

['--C', '0.2744966601103044', '--max_iter', '200']
*******************************************************************************
<bound method Run.log_list of Run(Experiment: udacity-aiml-project2,
Id: HD_4dfd0b48-7888-4cb4-b340-c1019c51a2a6_1,
Type: azureml.scriptrun,
Status: Completed)>


In [16]:
best_model = best_run.register_model

In [17]:
import joblib

if ("outputs" not in os.listdir()): 
    os.mkdir("outputs")

os.chdir("outputs")

# joblib.dump(best_run, "sklearn_regression_model.pkl")

In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

ds = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv', validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')
print(ds.to_pandas_dataframe())

       age          job  marital          education  default housing loan  \
0       57   technician  married        high.school       no      no  yes   
1       55      unknown  married            unknown  unknown     yes   no   
2       33  blue-collar  married           basic.9y       no      no   no   
3       36       admin.  married        high.school       no      no   no   
4       27    housemaid  married        high.school       no     yes   no   
...    ...          ...      ...                ...      ...     ...  ...   
32945   56    housemaid  married           basic.4y       no      no  yes   
32946   37   management  married  university.degree       no      no  yes   
32947   26       admin.   single  university.degree       no      no   no   
32948   31  blue-collar   single           basic.9y       no      no   no   
32949   39    housemaid  married           basic.4y       no      no   no   

         contact month day_of_week  ...  campaign  pdays  previous  \
0    

In [19]:
from trainudacityazureml import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [44]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_hours=30,
    compute_target= cpu_cluster,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=10)

In [45]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-aiml-project2,AutoML_454382e9-089c-424d-9f0f-11962e5b2dbc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [46]:
automl_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-aiml-project2,AutoML_454382e9-089c-424d-9f0f-11962e5b2dbc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the t

{'runId': 'AutoML_454382e9-089c-424d-9f0f-11962e5b2dbc',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-06-08T18:29:24.136408Z',
 'endTimeUtc': '2022-06-08T19:16:44.430498Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '10',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-aiml-project2","subscription_id":"16bc73b5-82be-47f2-b5ab-f2373344794c","resource_group":"epe-poc-nazeer","workspace_name":"nahmed30-azureml-workspace","region":"centralus","compute_target":"cpu-cluster","sp

In [47]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, fitted_model = automl_run.get_output()
print(best_automl_run)

Run(Experiment: udacity-aiml-project2,
Id: AutoML_454382e9-089c-424d-9f0f-11962e5b2dbc_30,
Type: azureml.scriptrun,
Status: Completed)


In [48]:
if ("outputs" not in os.listdir()): 
    os.mkdir("outputs")

os.chdir("outputs")
joblib.dump(fitted_model,"AUTOML_BEST_MODEL.pkl")

['AUTOML_BEST_MODEL.pkl']